## Fura API

https://github.com/fura-protocol/fura-api-quickswap

### Quickswap

14/09/22

In [1]:
import pandas as pd
import numpy as np
import time
import requests
from json import JSONDecodeError
import datetime as dt

In [2]:
endpoint = 'https://api.fura.org/subgraphs/name/quickswap'

In [16]:
#Check if subgraph is up to date
query = """{
    blocks(first: 10, orderBy: number, orderDirection: desc){
        number
        timestamp
    }
}"""

request = requests.post(endpoint, json={'query':query})
data = request.json()
#Check if it is ok to run i.e. within 5 days of today
print(dt.datetime.fromtimestamp(int(data['data']['blocks'][0]['timestamp'], base = 0)))
int(data['data']['blocks'][0]['timestamp'], base = 0) > (dt.datetime.today().timestamp() - 3600*24*5)

2022-09-14 09:31:48


True

6-Month Average Daily Volume

In [20]:
query = """{
  uniswapDayDatas(first: 183, orderBy: date, orderDirection: desc){
    id
    date
    dailyVolumeUSD
    totalVolumeUSD
    totalLiquidityUSD
    txCount
  }
}
"""

In [21]:
request = requests.post(endpoint, json={'query':query})
data = request.json()

In [22]:
data

{'data': {'uniswapDayDatas': [{'id': '19249',
    'date': 1663113600,
    'dailyVolumeUSD': '4209894.6240258089976946',
    'totalVolumeUSD': '0.0000000000000000',
    'totalLiquidityUSD': '243656098.8005927569454542',
    'txCount': '126797015'},
   {'id': '19248',
    'date': 1663027200,
    'dailyVolumeUSD': '22408536.6902110435092405',
    'totalVolumeUSD': '0.0000000000000000',
    'totalLiquidityUSD': '244823714.2468509205811127',
    'txCount': '126768987'},
   {'id': '19247',
    'date': 1662940800,
    'dailyVolumeUSD': '17901076.7870023795193636',
    'totalVolumeUSD': '0.0000000000000000',
    'totalLiquidityUSD': '268048444.0837787276493884',
    'txCount': '126675208'},
   {'id': '19246',
    'date': 1662854400,
    'dailyVolumeUSD': '10103727.2442941236904233',
    'totalVolumeUSD': '0.0000000000000000',
    'totalLiquidityUSD': '276755640.2010968982144749',
    'txCount': '126575016'},
   {'id': '19245',
    'date': 1662768000,
    'dailyVolumeUSD': '11457542.52962529527

In [23]:
df = pd.json_normalize(data['data']['uniswapDayDatas'])

In [25]:
df

,id,date,dailyVolumeUSD,totalVolumeUSD,totalLiquidityUSD,txCount
0,19249,1663113600,4209894.6240258089976946,0.0000000000000000,243656098.8005927569454542,126797015
1,19248,1663027200,22408536.6902110435092405,0.0000000000000000,244823714.2468509205811127,126768987
2,19247,1662940800,17901076.7870023795193636,0.0000000000000000,268048444.0837787276493884,126675208
3,19246,1662854400,10103727.2442941236904233,0.0000000000000000,276755640.2010968982144749,126575016
4,19245,1662768000,11457542.5296252952739107,0.0000000000000000,279824998.1910243848462749,126502544
...,...,...,...,...,...,...
178,19071,1647734400,39019542.1412451269922445,0.0000000000000000,791174383.9495744011138927,103152482
179,19070,1647648000,47476175.7317139641481310,0.0000000000000000,811700438.1794869286112890,103008173
180,19069,1647561600,31945149.9637772542739050,0.0000000000000000,799743400.8341705345937075,102858606
181,19068,1647475200,37529201.9966315297683163,0.0000000000000000,789324620.9841030298687954,102751167


In [26]:
df['dailyVolumeUSD'].astype(float).mean()

31804149.409806445

6-Month Average TVL

In [27]:
#DefiLlama https://defillama.com/protocols/dexes 
lma = pd.read_csv("https://api.llama.fi/dataset/quickswap.csv")

/tmp/ipykernel_143337/2541364275.py:2: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,26

In [28]:
lma

,Unnamed: 0,Date,Timestamp,Quickswap,Quickswap.1,Quickswap.2,Quickswap.3,Quickswap.4,Quickswap.5,Quickswap.6,...,Quickswap.3832,Quickswap.3833,Quickswap.3834,Quickswap.3835,Quickswap.3836,Quickswap.3837,Quickswap.3838,Quickswap.3839,Quickswap.3840,Quickswap.3841
0,NaN,NaN,NaN,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,...,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes
1,NaN,NaN,NaN,Total,polygon,Total,Total,Total,Total,Total,...,polygon,polygon,polygon,polygon,polygon,polygon,polygon,polygon,polygon,polygon
2,NaN,NaN,NaN,TVL,TVL,Tokens(USD),Tokens(USD),Tokens(USD),Tokens(USD),Tokens(USD),...,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens
3,NaN,NaN,NaN,NaN,NaN,USDT,WETH,DAI,UNKNOWN (polygon:0xa1c09c8f4f5d03fcc27b456475d...,QUICK,...,FEVR,CROWD,SMBR,SPIT,ZKP,PAPER,TOTEM,PS1,ROGUE,VERSE
4,NaN,09/10/2020,1.602202e+09,444.9387448941156,444.9387448941156,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692,NaN,10/09/2022,1.662768e+09,279931006.241759,279931006.241759,279931006.241759,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
693,NaN,11/09/2022,1.662854e+09,279968810.764994,279968810.764994,279968810.764994,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
694,NaN,12/09/2022,1.662941e+09,275025990.730388,275025990.730388,275025990.730388,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
695,NaN,13/09/2022,1.663027e+09,266352300.113199,266352300.113199,266352300.113199,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
lma.iloc[:,3]

0                  Dexes
1                  Total
2                    TVL
3                    NaN
4      444.9387448941156
             ...        
692     279931006.241759
693     279968810.764994
694     275025990.730388
695     266352300.113199
696     245583018.385664
Name: Quickswap, Length: 697, dtype: object

In [30]:
#Just eth currently can take total by changing the header
lma.iloc[-183:, 3].astype(float).mean()

476075465.5228742

Largest Pool Concentration

In [40]:
query = """{
  pairs(first: 1000, orderBy: createdAtTimestamp, orderDirection: asc, where: {createdAtTimestamp_gt: "0"}){
    id
    token0{
      symbol
      derivedETH
    }
    token1{
      symbol
      derivedETH
    }
    reserve0
    reserve1
    reserveUSD
    createdAtTimestamp
  }
}
"""

In [41]:
request = requests.post(endpoint, json={'query':query})
data = request.json()

In [42]:
pd.json_normalize(data['data']['pairs'])

,id,reserve0,reserve1,reserveUSD,createdAtTimestamp,token0.symbol,token0.derivedETH,token1.symbol,token1.derivedETH
0,0x853ee4b2a13f8a742d64c8f088be7ba2131f670d,6337828.603112000000000000,3958.971733400769028355,12675657.2062234068326512,1602189626,USDC,0.0006246574310099,WETH,1.0000000000000000
1,0x4a35582a710e1f4b2030a3f826da20bfb6703c09,1438.831093901771851451,2304749.330510572983441775,4607747.1334704377163009,1602195658,WETH,1.0000000000000000,DAI,0.0006242896244092
2,0x6e7a5fafcec6bb1e78bae2a1f0b612012bf14827,8492786.542402020598742172,7286092.396996000000000000,14559629.8372331797082654,1602199160,WMATIC,0.0005349797959819,USDC,0.0006246574310099
3,0x604229c960e5cacf2aaeac8be68ac07ba9df81c3,2865037.465729874259504782,2456818.473796000000000000,4910611.1817505934957105,1602206700,WMATIC,0.0005349797959819,USDT,0.0006254100228413
4,0x59153f27eefe07e5ece4f9304ebba1da6f53ca88,93753.539776518779126004,93833.868272000000000000,187779.5868220792651081,1602214500,DAI,0.0006242896244092,USDT,0.0006254100228413
...,...,...,...,...,...,...,...,...,...
995,0x801175fb1ed1ec437745a37901349409d2bdba55,0.000025002691615221,0.000024554841410689,0.0000670205870794,1619627476,WMATIC,0.0005349797959819,PIX,0.0000000000000000
996,0xb95dd2581dd2abfa0b8d0e53c0855cec1a3fb323,2.130221655350416159,4609.169331830633281553,2.3323543142013177,1619630658,WMATIC,0.0005349797959819,MTT,0.0000000000000000
997,0x322f956de1379b81cb9b744aab9fc34199ed1959,0.999998992828084021,0.009801009901004925,0.0000000000000000,1619631998,dm,0.0000000000000000,axM,0.0000000000000000
998,0x2ed3a5909c1e0f5fd47f00c7ed7d0815ff766b12,0.000000000000000000,0.000000000000000000,0.0000000000000000,1619632098,dm,0.0000000000000000,DG,0.0000000000000000


In [45]:

df = pd.DataFrame()
ts = 0

while True:
    sl = len(df)
    query = """{
  pairs(first: 1000, orderBy: createdAtTimestamp, orderDirection: asc, where: {createdAtTimestamp_gt: """+f'"{ts}"'+"""}){
    id
    token0{
      symbol
      derivedETH
    }
    token1{
      symbol
      derivedETH
    }
    reserve0
    reserve1
    reserveUSD
    createdAtTimestamp
  }
}
"""
    request = requests.post(endpoint, json={'query':query})
    if request.ok:
      data = request.json()
      pool = pd.json_normalize(data['data']['pairs'])
      df = df.append(pool)
      df.drop_duplicates(inplace=True)
      if len(df) == sl:
        break
      ts = df['createdAtTimestamp'].max()
    

/tmp/ipykernel_143337/1461651834.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pool)
/tmp/ipykernel_143337/1461651834.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pool)
/tmp/ipykernel_143337/1461651834.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pool)
/tmp/ipykernel_143337/1461651834.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pool)
/tmp/ipykernel_143337/1461651834.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pool)
/tmp/ipykernel_14333

In [46]:
df

,id,reserve0,reserve1,reserveUSD,createdAtTimestamp,token0.symbol,token0.derivedETH,token1.symbol,token1.derivedETH
0,0x853ee4b2a13f8a742d64c8f088be7ba2131f670d,6338771.285579000000000000,3958.384733401379055666,12675660.1736251574903517,1602189626,USDC,0.0006244719291903,WETH,1.0000000000000000
1,0x4a35582a710e1f4b2030a3f826da20bfb6703c09,1439.026914616658774516,2304436.644286519755207495,4606785.5097808646322915,1602195658,WETH,1.0000000000000000,DAI,0.0006244593090396
2,0x6e7a5fafcec6bb1e78bae2a1f0b612012bf14827,8490755.213099768356711457,7287844.068101000000000000,14565258.6699341560679267,1602199160,WMATIC,0.0005357822192230,USDC,0.0006244719291903
3,0x604229c960e5cacf2aaeac8be68ac07ba9df81c3,2863383.741245806657671185,2458241.659939000000000000,4914354.7473542229631941,1602206700,WMATIC,0.0005357822192230,USDT,0.0006243443098892
4,0x59153f27eefe07e5ece4f9304ebba1da6f53ca88,93753.539776518779126004,93833.868272000000000000,187779.5868220792651081,1602214500,DAI,0.0006244593090396,USDT,0.0006243443098892
...,...,...,...,...,...,...,...,...,...
148,0x112bd085386e638685ea1c5021c3641190c1c1ff,340.156918316254880577,290.385394828352976767,0.0000000000000000,1663136539,UNI-V2,0.0000000000000000,Dogma 888,0.0000000000000000
149,0x2d9ec0d332206abbf03a900123d9af514af5b870,3.615000000000000000,69.990000000000000000,0.1048282428878953,1663138557,WDTH,0.0000000000000000,TEL,0.0000009321751093
150,0xb04e52c8aec5bdb362113c3e3c0248650996906e,0.000075003184232663,0.066763904223556982,0.0000643542387954,1663138673,WMATIC,0.0005358071851531,DMETA,0.0000000000000000
151,0xf6327790d76e05877cb418b4b61d6eb2c853c3be,18608.545974738898839292,4.300000000000000000,3.6918341194638175,1663141594,NKTM,0.0000000000000000,WMATIC,0.0005358071851531


In [56]:
df['reserveUSD'] = df['reserveUSD'].astype(float)
df.sort_values('reserveUSD', inplace=True)

In [58]:
df.reset_index(drop = True, inplace=True)

In [68]:
df.loc[df['reserveUSD'] >= 14565258.6699341560679267]

,id,reserve0,reserve1,reserveUSD,createdAtTimestamp,token0.symbol,token0.derivedETH,token1.symbol,token1.derivedETH
37924,0x6e7a5fafcec6bb1e78bae2a1f0b612012bf14827,8490755.213099768356711457,7287844.068101000000000000,1.456526e+07,1602199160,WMATIC,0.0005357822192230,USDC,0.0006244719291903
37925,0x160532d2536175d65c03b97b0630a9802c274dad,8736006.543832000000000000,8818705.240890145260904442,1.749822e+07,1620019955,USDC,0.0006244719291903,miMATIC,0.0006206559182530
37926,0x369582d2010b6ed950b571f4101e3bb9b554876f,19274274.639661560898038453,18657328.715037214632642196,3.307369e+07,1638865146,WMATIC,0.0005358703411179,SAND,0.0005534972630997
37927,0xac9cb2b5baf2bf0ba1103872ddd7f921edf2824b,0.309348023667226136,9899999956.465768665631464985,6.490422e+07,1658997893,WMATIC,0.0005358071851531,WOMBAT,0.0000037762568106
37928,0x6189e9da138baa5226e133a8e9a3edd7ad5ab810,0.000080584904947016,1000000047466.450635835358340490,5.659946e+10,1643242243,WMATIC,0.0005358703411179,TITAN,0.0000230633233676
...,...,...,...,...,...,...,...,...,...
38148,0x9b6dbbf2a40b82e83d604ff382e07ee1fff0dde9,0.000004537564598911,1000000000223.800870186249509043,1.460538e+14,1645295812,WMATIC,0.0005358703411179,DEFIATO,0.0530802667989735
38149,0xa85f62f01db0d0c967bfaaa5cf073dc658ab5651,0.000003026480510937,1000000000202.963948209529434044,1.477925e+14,1644421065,WMATIC,0.0005358703411179,Alpha Centauri,0.0460466852066374
38150,0xf745299ee96d91d1b53d2d0e7ef3d89e7287407b,0.000003026470652832,1000000000202.995312027626475247,1.478289e+14,1644271163,WMATIC,0.0005358703411179,DISNEY,0.0470104767002101
38151,0x75c8bf6df135e37dff6488cba47845d28908d4e4,0.000003026439481920,1000000000202.523558519937656744,1.490268e+14,1644264071,WMATIC,0.0005358703411179,Alpha Centauri,0.0473604955916699


In [70]:
df = df.iloc[:37927].copy()

In [71]:
df['reserveUSD'].astype(float).max()/df['reserveUSD'].astype(float).sum()

0.12862312058152103

In [72]:
df['reserveUSD'].astype(float).nlargest(10).sum()/df['reserveUSD'].astype(float).sum()

0.4888087910741003

Liquidity Pools with > $1mn TVL

In [74]:
len(df.loc[df['reserveUSD'].astype(float) > 1000000])

36

Monthly Active Users

In [21]:
query = """{
  usageMetricsDailySnapshots(first: 183, orderBy: timestamp, orderDirection: desc){
    timestamp
    blockNumber
    id
    dailyActiveUsers
    dailySwapCount
    dailyDepositCount
    dailyWithdrawCount 
  }
}
"""

In [22]:
request = requests.post(endpoint, json={'query':query})
data = request.json()

In [23]:
df = pd.json_normalize(data['data']['usageMetricsDailySnapshots'])

In [24]:
df['date'] = df['timestamp'].astype(int).apply(lambda x: dt.datetime.fromtimestamp(x))

In [25]:
monthactiveusers = []
for i in range(len(df['date'].dt.month.unique())):
    monthactiveusers.append(df['dailyActiveUsers'].loc[df['date'].dt.month == df['date'].dt.month.unique()[i]].astype(float).sum())

In [26]:
np.mean(monthactiveusers) #can chart this easy

40666.71428571428

Implied slippage

In [75]:
#Can comeback with enough time